<a href="https://colab.research.google.com/github/Moorea-AI/ANALITICA3/blob/main/%5CRH_Grupo1%5Cb_preprocesamiento.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [45]:
def ejecutar_sql (nombre_archivo, cur):
  sql_file=open(nombre_archivo)
  sql_as_string=sql_file.read()
  sql_file.close
  cur.executescript(sql_as_string)

In [46]:
#### Cargar paquetes
import pandas as pd ### para manejo de datos
import sqlite3 as sql #### para bases de datos sql
import sys ## saber ruta de la que carga paquetes


In [47]:
#se generan las url para conectar las bases de datos desde git hub
employee = 'https://raw.githubusercontent.com/Moorea-AI/ANALITICA3/main/RH_Grupo1/databases/employee_survey_data.csv'
general = 'https://raw.githubusercontent.com/Moorea-AI/ANALITICA3/main/RH_Grupo1/databases/general_data.csv'
manager = 'https://raw.githubusercontent.com/Moorea-AI/ANALITICA3/main/RH_Grupo1/databases/manager_survey.csv'
retirement = 'https://raw.githubusercontent.com/Moorea-AI/ANALITICA3/main/RH_Grupo1/databases/retirement_info.csv'

In [48]:
# se leen los archivos csv
df_employee=pd.read_csv(employee)
df_general=pd.read_csv(general)
df_manager=pd.read_csv(manager)
df_retirement=pd.read_csv(retirement)

In [49]:
# resumen de información de las tablas
df_employee.info()
df_general.info()
df_manager.info()
df_retirement.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8820 entries, 0 to 8819
Data columns (total 6 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Unnamed: 0               8820 non-null   int64  
 1   EmployeeID               8820 non-null   int64  
 2   EnvironmentSatisfaction  8770 non-null   float64
 3   JobSatisfaction          8780 non-null   float64
 4   WorkLifeBalance          8744 non-null   float64
 5   DateSurvey               8820 non-null   object 
dtypes: float64(3), int64(2), object(1)
memory usage: 413.6+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8820 entries, 0 to 8819
Data columns (total 25 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Unnamed: 0               8820 non-null   int64  
 1   Age                      8820 non-null   int64  
 2   BusinessTravel           8820 non-null   object 
 3   Department      

In [50]:
#### Convertir campos a formato fecha
df_employee["DateSurvey"]=pd.to_datetime(df_employee['DateSurvey'], format="%Y/%m/%d")
df_general["InfoDate"]=pd.to_datetime(df_general['InfoDate'], format="%Y/%m/%d")
df_manager["SurveyDate"]=pd.to_datetime(df_manager['SurveyDate'], format="%Y/%m/%d")
df_retirement["retirementDate"]=pd.to_datetime(df_retirement['retirementDate'], format="%Y/%m/%d")

In [51]:
## eliminar columnas
df_employee =df_employee.drop(["Unnamed: 0"], axis=1)
df_general =df_general.drop(["Unnamed: 0"], axis=1)
df_manager =df_manager.drop(["Unnamed: 0"], axis=1)
df_retirement =df_retirement.drop(["Unnamed: 0","Unnamed: 0.1"], axis=1)

In [52]:
### por facilidad se convierten los nombres de las columnas a minuscula
df_employee = df_employee.rename(columns=lambda x: x.lower())
df_general = df_general.rename(columns=lambda x: x.lower())
df_manager = df_manager.rename(columns=lambda x: x.lower())
df_retirement = df_retirement.rename(columns=lambda x: x.lower())

In [53]:
df_employee.head(5)

,employeeid,environmentsatisfaction,jobsatisfaction,worklifebalance,datesurvey
0,1,3.0,4.0,2.0,2015-12-31
1,2,3.0,2.0,4.0,2015-12-31
2,3,2.0,2.0,1.0,2015-12-31
3,4,4.0,4.0,3.0,2015-12-31
4,5,4.0,1.0,3.0,2015-12-31


In [54]:
#crear base de datos en SQL
conn= sql.connect("data\\db_empleados") ### crea una base de datos con el nombre dentro de comillas, si existe crea una conexión.
cur=conn.cursor() ### ejecutar funciones  en BD


In [55]:
### Llevar tablas a base de datos
df_employee.to_sql("employee",conn,if_exists="replace")
df_general.to_sql("general",conn,if_exists="replace")
df_manager.to_sql("manager",conn,if_exists="replace")
df_retirement.to_sql("retirement",conn,if_exists="replace")

711

In [56]:
cur.execute("Select name from sqlite_master where type='table'") ### consultar bases de datos
cur.fetchall()

[('employee',), ('general',), ('manager',), ('retirement',)]

In [57]:
pd.read_sql("""select employeeid,count(*)
                            from employee
                            group by employeeid""", conn)

,employeeid,count(*)
0,1,2
1,2,2
2,3,2
3,4,2
4,5,2
...,...,...
4405,4406,2
4406,4407,2
4407,4408,2
4408,4409,2
